In [3]:
import numpy as np
import xarray as xr
import gsw

In [51]:
# Define dimensions and coordinates
dims = ['y','z','t']
# 2x2x2
y = np.arange(0,2)
z = np.arange(0,2)
t = np.arange(0,2)
# Define numpy arrays of salinity, temperature and pressure
SA_vals = np.array([[[34.7,34.8],[34.9,35]],[[35.1,35.2],[35.3,35.4]]])
CT_vals = np.array([[[7,8],[9,10]],[[11,12],[13,14]]])
p_vals = np.array([10,20])
lat_vals = np.array([0,10])
# Plug in to xarray objects
SA = xr.DataArray(SA_vals,dims=dims,coords={'y':y,'z':z,'t':t})
CT = xr.DataArray(CT_vals,dims=dims,coords={'y':y,'z':z,'t':t})
p = xr.DataArray(p_vals,dims=['z'],coords={'z':z})
lat = xr.DataArray(lat_vals,dims=['y'],coords={'y':y})

#### Test 1: Dimensions and coordinates match

In [37]:
out = gsw.sigma0(SA,CT)
out

array([[[27.05505314, 26.98944507],
        [26.91255169, 26.82464446]],

       [[26.72598005, 26.61680134],
        [26.49733843, 26.36780957]]])

In [38]:
out = xr.apply_ufunc(gsw.sigma0,
                    SA,CT)
out

<xarray.DataArray (x: 2, z: 2, t: 2)>
array([[[27.05505314, 26.98944507],
        [26.91255169, 26.82464446]],

       [[26.72598005, 26.61680134],
        [26.49733843, 26.36780957]]])
Coordinates:
  * x        (x) int64 0 1
  * z        (z) int64 0 1
  * t        (t) int64 0 1

#### Test 2: Broadcasting along dimension required (dimensions known)

In [52]:
out = gsw.alpha(SA,CT,p)
out

array([[[0.00013465, 0.00014573],
        [0.00015611, 0.00016666]],

       [[0.00017658, 0.00018666],
        [0.00019616, 0.00020582]]])

In [53]:
out = xr.apply_ufunc(gsw.alpha,
                    SA,CT,p)
out

<xarray.DataArray (y: 2, z: 2, t: 2)>
array([[[0.00013465, 0.00014551],
        [0.00015632, 0.00016666]],

       [[0.00017658, 0.00018647],
        [0.00019634, 0.00020582]]])
Coordinates:
  * y        (y) int64 0 1
  * z        (z) int64 0 1
  * t        (t) int64 0 1

#### Test 3: Broadcasting along dimension required (dimensions unknown/exclusive)

In [54]:
out = gsw.z_from_p(p,lat)
out

array([ -9.94483891, -19.88602889])

In [55]:
out = xr.apply_ufunc(gsw.z_from_p,
                    p,lat)
out

<xarray.DataArray (z: 2, y: 2)>
array([[ -9.94483891,  -9.94325586],
       [-19.88919491, -19.88602889]])
Coordinates:
  * z        (z) int64 0 1
  * y        (y) int64 0 1

### Now introduce chunks

In [65]:
SA_chunk = SA.chunk(chunks={'y':1,'t':1})
CT_chunk = CT.chunk(chunks={'y':1,'t':1})
lat_chunk = lat.chunk(chunks={'y':1})

#### Test 1: Dimensions and coordinates match

In [66]:
out = gsw.sigma0(SA_chunk,CT_chunk)
out

array([[[27.05505314, 26.98944507],
        [26.91255169, 26.82464446]],

       [[26.72598005, 26.61680134],
        [26.49733843, 26.36780957]]])

In [67]:
out = xr.apply_ufunc(gsw.sigma0,
                    SA_chunk,CT_chunk,
                    dask='parallelized',output_dtypes=['float32'])
out

<xarray.DataArray (y: 2, z: 2, t: 2)>
dask.array<sigma0, shape=(2, 2, 2), dtype=float32, chunksize=(1, 2, 1), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) int64 0 1
  * z        (z) int64 0 1
  * t        (t) int64 0 1

#### Test 2: Broadcasting along dimension required (dimensions known)

In [68]:
out = gsw.alpha(SA_chunk,CT_chunk,p)
out

array([[[0.00013465, 0.00014573],
        [0.00015611, 0.00016666]],

       [[0.00017658, 0.00018666],
        [0.00019616, 0.00020582]]])

In [69]:
out = xr.apply_ufunc(gsw.alpha,
                    SA_chunk,CT_chunk,p,
                    dask='parallelized',output_dtypes=['float32'])
out

<xarray.DataArray (y: 2, z: 2, t: 2)>
dask.array<alpha, shape=(2, 2, 2), dtype=float32, chunksize=(1, 2, 1), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) int64 0 1
  * z        (z) int64 0 1
  * t        (t) int64 0 1

#### Test 3: Broadcasting along dimension required (dimensions unknown/exclusive)

In [70]:
out = gsw.z_from_p(p,lat_chunk)
out

array([ -9.94483891, -19.88602889])

In [73]:
out = xr.apply_ufunc(gsw.z_from_p,
                    p,lat_chunk,
                    dask='parallelized',output_dtypes=['float32'])
out

<xarray.DataArray (z: 2, y: 2)>
dask.array<z_from_p, shape=(2, 2), dtype=float32, chunksize=(2, 1), chunktype=numpy.ndarray>
Coordinates:
  * z        (z) int64 0 1
  * y        (y) int64 0 1